Using https://www.kaggle.com/cyannani123/keras-cellular-image-classification as test example

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB1
from tensorflow.python.client import device_lib
import tensorflow.keras as keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, concatenate, Convolution2D, MaxPool2D, Flatten
from tensorflow.keras.utils import Sequence
import os, sys, random,copy
import matplotlib.pyplot as plt

from PIL import Image
from PIL import ImageFilter
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
from tqdm import tqdm

In [3]:
sys.path.append('../src')

In [4]:
from data_v2 import augment
from data_v2 import get_center_box
from data_v2 import get_random_subbox
from data_v2 import ImgGen

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2021-11-21 15:46:02.395613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-21 15:46:02.423815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-21 15:46:02.424170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.


Loading test and train data.

Will load directly from zip for now.

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, concatenate, Convolution2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers

In [12]:
def create_cnn_model(learning_rate=0.001, output_classes=1, input_shape=(6,224,224)):
    """
    CNN model
    """

    input_data = Input(name="input_image",shape=input_shape)

    cnn_seq = Sequential([
        Convolution2D(filters=64, kernel_size=(3,3), input_shape=(6,224,224), activation='relu', padding='same',data_format='channels_first'),
        Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D((2,2)),
        Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D((2,2)),
        Convolution2D(filters=16, kernel_size=(3,3), activation='relu', padding='same'),
        Convolution2D(filters=16, kernel_size=(3,3), activation='relu', padding='same'),
        Flatten(),
        Dense(units=8192, activation="relu"),
        Dense(units=4096, activation="relu"),
        Dense(units=1108, activation="softmax")
    ])

    cnn_model = cnn_seq(input_data)

    model = Model(inputs=[input_data], outputs=cnn_model)

    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])
    model.summary()

    return model

In [13]:
model = create_cnn_model(output_classes=len(train_gen.label_encoder.classes_))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 6, 224, 224)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 1108)              155713972 
Total params: 155,713,972
Trainable params: 155,713,972
Non-trainable params: 0
_________________________________________________________________


2021-11-21 15:48:13.452185: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-21 15:48:13.477578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-21 15:48:13.480081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-21 15:48:13.480294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may hav

Training model

In [11]:
test_size = 0.025
batch_size = 32

In [14]:
print(f"Training set batched size {len(train_gen)}")
print(f"Validation set batched size {len(val_gen)}")

Training set batched size 56
Validation set batched size 2


In [15]:
filepath = 'ModelCheckpoint_all.h5'

In [17]:
callback = [
        ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
        ]

In [19]:
history = model.fit(train_gen, 
                              steps_per_epoch=len(train_gen), 
                              epochs=100, 
                              verbose=1, 
                              validation_data=val_gen,
                              validation_steps=len(val_gen),
                              callbacks=callback
                             )

Epoch 1/100
56/56 [==============================] - 3s 50ms/step - loss: 6.6897 - accuracy: 0.0045 - val_loss: 7.4530 - val_accuracy: 0.0217

Epoch 00001: val_loss did not improve from 6.99965
Epoch 2/100
56/56 [==============================] - 2s 43ms/step - loss: 6.6915 - accuracy: 0.0045 - val_loss: 7.4406 - val_accuracy: 0.0217

Epoch 00002: val_loss did not improve from 6.99965
Epoch 3/100
56/56 [==============================] - 3s 49ms/step - loss: 6.6880 - accuracy: 0.0045 - val_loss: 7.4624 - val_accuracy: 0.0217

Epoch 00003: val_loss did not improve from 6.99965
Epoch 4/100
56/56 [==============================] - 2s 43ms/step - loss: 6.6839 - accuracy: 0.0045 - val_loss: 7.4409 - val_accuracy: 0.0217

Epoch 00004: val_loss did not improve from 6.99965
Epoch 5/100
56/56 [==============================] - 3s 50ms/step - loss: 6.6839 - accuracy: 0.0045 - val_loss: 7.5110 - val_accuracy: 0.0217

Epoch 00005: val_loss did not improve from 6.99965
Epoch 6/100
56/56 [===========

In [23]:
history_df = pd.DataFrame(history.history)

In [24]:
history_df

,loss,accuracy,val_loss,val_accuracy
0,6.689732,0.004494,7.452999,0.021739
1,6.691470,0.004494,7.440597,0.021739
2,6.688009,0.004494,7.462411,0.021739
3,6.683939,0.004494,7.440946,0.021739
4,6.683856,0.004494,7.510971,0.021739
...,...,...,...,...
95,6.662679,0.004494,7.946043,0.021739
96,6.660921,0.004494,7.965796,0.021739
97,6.662498,0.004494,7.894928,0.021739
98,6.661799,0.004494,7.959733,0.021739
